In [45]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
import jax

import scipy.stats as stats
from scipy.optimize import minimize

import autograd.numpy as auto_np
from autograd import hessian

from utils import is_positive_definite, flat_sigma_to_sigma_matrix, sigma_matrix_to_flat_sigma
from init import init

s, w, x, y, z, sigma_matrix, beta, g_0, G_0, beta_0_init, B_0 = init()
gamma, beta_0, beta_1 = beta[:5], beta[5:9], beta[9:]

In [46]:
def log_likelyhood_ys(w, x, s, z, gamma, beta_0, beta_1, sigma_matrix):
    
    log_prod = 0
    for i in range(len(s)):
        treatment = s[i]
        if treatment == 0:
            fn = stats.norm.pdf(z[0][i],  loc=np.dot(x[i], beta_0), 
                                scale=sigma_matrix[1][1]**2)
            normal_cdf = stats.norm.cdf((np.dot(-w[i], gamma) - (sigma_matrix[0][1]/sigma_matrix[1][1])
                                         * (z[0][i] - np.dot(x[i], beta_0)))
                                         / (1-sigma_matrix[0][1]**2/sigma_matrix[1][1])**0.5)
        else:
            fn = stats.norm.pdf(z[1][i],  loc=np.dot(x[i], beta_1), 
                                scale=sigma_matrix[2][2]**2)
            normal_cdf = stats.norm.cdf((np.dot(-w[i], gamma) - (sigma_matrix[0][2]/sigma_matrix[2][2])
                                         * (z[1][i] - np.dot(x[i], beta_1)))
                                         / (1-sigma_matrix[0][2]**2/sigma_matrix[2][2])**0.5)
                    
        log_prod += np.log(max(fn * normal_cdf, 10**-300))

    return log_prod

def log_g_sigma(w, s, x, z, beta_0, beta_1, gamma, flat_sigma, g_0, G_0):

    sigma_matrix = flat_sigma_to_sigma_matrix(sigma_matrix)
    # likelihood of ys given sigma
    first_term = log_likelyhood_ys(w, x, s, z, gamma, beta_0, beta_1, sigma_matrix)

    # density of sigma given g_0, G_0
    second_term = np.log(stats.multivariate_normal.pdf(flat_sigma, g_0, G_0))

    third_term = 0 if is_positive_definite(sigma_matrix) else 0
    

    result = first_term + second_term + third_term
    return result

def q_sigma(w, s, x, z, beta_0, beta_1, gamma, sigma_matrix, g_0, G_0, sample=True):

    def sub_log_g_sigma(sigma_matrix):
        return log_g_sigma(w, s, x, z, beta_0, beta_1, gamma, sigma_matrix, g_0, G_0) 

    sigma_flat = sigma_matrix_to_flat_sigma(sigma_matrix)
    H = jax.hessian(sub_log_g_sigma)(sigma_flat)

    V = np.linalg.inv(-H)

    # Compute mu the mode of sub_log_g_sigma
    result = minimize(-log_g_sigma, x0=sigma_matrix)
    mode = result.x[0]

    if sample:
        # sample from multivariate t density
        return stats.multivariate_t.rvs(loc=mode, shape=V)
    else:
        # return likelyhood of sigma given mode and V
        return stats.multivariate_t.pdf(sigma_matrix, mode, V)

q_sigma(w, s, x, z, beta_0, beta_1, gamma, sigma_matrix, g_0, G_0, sample=False)

IndexError: Too many indices for array: 1 non-None/Ellipsis indices for dim 0.

In [44]:


def x_square(x):
    return x**2

jax.hessian(x_square)(np.array([1., 2.]))

DeviceArray([[[2., 0.],
              [0., 0.]],

             [[0., 0.],
              [0., 2.]]], dtype=float32)

TypeError: grad_np_sum() got an unexpected keyword argument 'out'